# SUB-BENCHMARK 2: Comparison of the jDR methos on cancer datasets

We here reproduce the second sub-benchmark proposed in the paper. We thus compare here the performances of the various jDR methods on the integration of multi-omics cancer datasets from TCGA. The methods are evaluated regarding associations to clinical annotations, survival and biological annotations (GO, REACTOME, Hallmarks).

## Comparison based on clinical annotations 基于临床注释的比较

We define here a function that performs the jDR comparison based on their association to clinical annotations. In particular, we here compute the number of clinical annotations significantly associated with at least one factor, and their selectivity (see Methods paper). The clinical metadata need to be stored in a clinical folder containing files named according to each cancer type. 
我们在这里定义了一个函数，它根据jDR与临床注释的关联来执行jDR比较。特别是，我们在这里计算与至少一个因素显著相关的临床注释的数量，以及它们的选择性（见方法论文）。临床元数据需要存储在临床文件夹中，其中包含根据每种癌症类型命名的文件。

In [1]:
## Perform clinical annotation-based comparison 
## INPUTS:
# factorizations = already computed factirizations
# clinical = clinical information associated with samples
# col = columns in clinical data on which the analysis will be performed
## OUPUTS: a list containing output values
# selectivity = Selectivity (fraction of significant annotations per all significant factors)
# nonZeroFacs = Number of unique factors that were significant at least once
# total_pathways = Number of clinical annotations with at least one significant factor component
##执行基于临床注释的比较
##输入：
#factorizations=已计算的因式分解
#clinical=与样本相关的临床信息
#col=将对其执行分析的临床数据列
##输出：包含输出值的列表
#selectivity=选择性（每个重要因素的重要注释分数）
#nonZeroFacs=至少一次显著的唯一因素的数量
#total_pathways=至少有一个显著因子成分的临床注释数
clinical_comparison <- function(factorizations, clinical, col){

    # Empty containers
    line <- numeric(0)
    line2 <- numeric(0)
    line3 <- numeric(0)

    # For each factorization
    for(i in 1:length(factorizations)){

        # Extract sample association
        factors <- factorizations[[i]][[1]]
        if(is.null(names(factors))) { names(factors) <- 1:ncol(factors) }
        if(is.null(colnames(factors))) { colnames(factors) <- 1:ncol(factors) }

        # Patient names in factorisation results
        patient.names <- rownames(factors)
        # Patient names in original data
        patient.names.in.file <- as.character(clinical[, 1])
        patient.names.in.file <- toupper(gsub('-', '\\.', patient.names.in.file))
        # Remove non-matching patient names
        is_in_file <- patient.names %in% patient.names.in.file
        if(length(patient.names)!=sum(is_in_file)) {
            factors <- factors[is_in_file, ]
            patient.names <- patient.names[is_in_file]
            rownames(factors)<-patient.names
        }
        # Match indices of patient names
        indices <- match(patient.names, patient.names.in.file)
        # Use indices to extract coresponding survival information
        ordered.clinical.data <- clinical[indices,]        
        
        # Only use column names that are really present in the data
        col_new <- col[col %in% colnames(ordered.clinical.data)]

        # Stor all p-values
        pvalues <- numeric(0)
        # Store number of significant annotations
        clin_erich <- 0 
        
        # Test significance association with clinical annotations
        for(j in col_new){
            
            # Perform the analysis if there is more than one possible value in current column
            table_values <- table(ordered.clinical.data[,j])
            if(sum(table_values>0)>1){

                if(j == "age_at_initial_pathologic_diagnosis" ){
                    pvalues_col <- apply(factors, MARGIN=2, 
                                       function(x) kruskal.test(x~cut(as.numeric(ordered.clinical.data[,j]),
                                                                      5, include.lowest=TRUE))$p.value)
                    pvalues <- c(pvalues, pvalues_col)
                    if(min(pvalues_col)<0.05){
                        clin_erich <- clin_erich+1
                    }
                }else if(j == "days_to_new_tumor_event_after_initial_treatment"){
                    pvalues_col <- apply(factors,MARGIN=2, 
                                       function(x) kruskal.test(x~cut(as.numeric(ordered.clinical.data[,j]), 
                                                                      3, include.lowest=TRUE))$p.value)
                    pvalues <- c(pvalues, pvalues_col)
                    if(min(pvalues_col)<0.05){
                        clin_erich <- clin_erich+1
                    }
                }else if(j == "gender" || j == "history_of_neoadjuvant_treatment"){
                    pvalues_col <- apply(factors, MARGIN=2, 
                                       function(x) wilcox.test(x~ordered.clinical.data[,j])$p.value)
                    pvalues <- c(pvalues, pvalues_col)
                    if(min(pvalues_col)<0.05){
                        clin_erich <- clin_erich+1
                    }
                }
            }
        }
        
        # Number of clinical annotations with at least one significant p-value
        line3 <- rbind(line3, clin_erich)

        # Total number of significant factors in all tested columns
        column <- names(pvalues)[pvalues<0.05]
        # Number of unique factors that were significant at least once
        line2<-rbind(line2, length(unique(column)))

        # Number of times a p-value was found significant
        signif <- length(column)
        f<-length(unique(column))                                 
        # Selectivity 
        if(signif!=0){
            line <- rbind(line,((clin_erich/signif)+(f/signif))/2)
        }else{
            line <- rbind(line,0)
        }

    }
    
    # Store and return results
    out <- data.frame(selectivity=line, nonZeroFacs=line2, total_pathways=line3)
#    print(out)
    return(out)
}

## Comparison based on survival predictions 基于生存预测的比较

We here define the function that compares the performances of the various jDR methods based on their association to survival (computed through Cox regression).
我们在这里定义了一个函数，它根据各种jDR方法与生存率的关系（通过Cox回归计算）来比较它们的性能。

In [2]:
library('survival')

## Perform survival annotation-based comparison 
## INPUTS:
# factorizations = already computed factirizations
# method = methods used for factorization
# survival = survival data associated to the cancer
# out.folder = folder where results will be written
# cancer = name of currently analysed cancer
## OUPUTS: a list containing output values

##执行基于注释的比较
##输入：
#factorizations=已计算的因式分解
#method=用于因子分解的方法
#survival=与癌症相关的生存数据
# out.folder文件夹=将写入结果的文件夹
#cancer=当前分析癌症的名称
##输出：包含输出值的列表
survival_comparison <- function(factorizations, method, survival, out.folder, cancer){
    
    # Initialize result containers
    factors_cancer <- numeric(0)
    surv_final <- numeric(0)
    
    # Adjust sample names for breast survival dataset
    if(cancer=="breast"){survival[,1] <- paste0(survival[,1],"-01")}
    
    # For each computed factorisation
    for(i in 1:length(factorizations)){

        # Extract sample factors 
        factors <- factorizations[[i]][[1]]

        # Patient names in factorisation results
        patient.names <- rownames(factors)
        # Patient names in original data
        patient.names.in.file <- as.character(survival[, 1])
        patient.names.in.file <- toupper(gsub('-', '\\.', patient.names.in.file))
        # Remove non-matching patient names
        is_in_file <- patient.names %in% patient.names.in.file
        if(length(patient.names)!=sum(is_in_file)) {
            factors <- factors[is_in_file, ]
            patient.names <- patient.names[is_in_file]
            rownames(factors)<-patient.names
        }
        # Match indices of patient names
        indices <- match(patient.names, patient.names.in.file)
        # Use indices to extract coresponding survival information
        ordered.survival.data <- survival[indices,]
        # Clean data (assign 0 to NAs)
        ordered.survival.data$Survival[is.na(ordered.survival.data$Survival)] <- 0
        ordered.survival.data$Death[is.na(ordered.survival.data$Death)] <- 0

        # Calculate coxph
        coxph_obj <- coxph(Surv(ordered.survival.data$Survival, ordered.survival.data$Death) ~ factors)
        # P-values (corrected by the number of methods)
        pvalues <- length(factorizations)*as.matrix(coef(summary(coxph_obj))[,5])

        # How many significant? 
        factors_cancer <- c(factors_cancer, sum(pvalues<0.05))
        # Store p-values
        surv_final <- cbind(surv_final, pvalues) 
    }
    # Keep -log10 of p-values
    surv_final<-(-log10(surv_final))
    # Plot survival pvalues for each cancer type separately
    png(file=paste0(out.folder, "survival_", cancer, ".png"), width = 15, height = 15, units = 'in', res = 200)
    matplot(1:length(method), t(surv_final), 
            col="black", pch=18, xlab="Method", ylab="Pvalues survival", xaxt="none", cex=1.5)
    abline(h = (-log10(0.05)), v=0, col="black", lty=3, lwd=3)
    axis(1, at=1:length(method), labels=colnames(surv_final)) 
    dev.off()

#    print(factors_cancer)
    return(factors_cancer)
}

## Comparison based on association to biological annotations (REACTOME, Hallmarks, GO)基于与生物注释的关联的比较（反应组、标志、GO）
We define here a function that performs the jDR comparison based on their association to biological annotations. In particular, we here compute the number of factors significantly associated with at least a biological annotation and their selectivity (see Methods paper). 在这里定义一个函数，该函数根据它们与生物注释的关联来执行jDR比较。特别是，我们在这里计算了与至少一个生物注释及其选择性显著相关的因子的数量（见方法论文）。 

In [3]:
library("fgsea", quietly = TRUE)

## Perform biological annotation-based comparison 
## INPUTS:
# factorizations = already computed factirizations
# path.database = path to a GMT annotation file
# pval.thr = p-value threshold (default to 0.05)
## OUPUTS: a list containing output values
# selectivity = Selectivity (fraction of significant annotations per all significant factors)
# nonZeroFacs = Number of unique factors that were significant at least once
# total_pathways = Number of clinical annotations with at least one significant factor component
##执行基于生物注释的比较
##输入：
#factorizations=已计算的因式分解
# path.database =GMT注释文件的路径
# pval.thr =p值阈值（默认为0.05）
##OUPUTS：包含输出值的列表
#selectivity=选择性（每个重要因素的重要注释分数）
#nonZeroFacs=至少一次显著的唯一因素的数量
#total_pathways=至少有一个显著因子成分的临床注释数
biological_comparison <- function(factorizations, path.database, pval.thr=0.05){
    
    # Load annotation database
    pathways <- gmtPathways(path.database)
    
    # Containers to report results
    report_number <- numeric(0)
    report_nnzero <- numeric(0)
    report_select <- numeric(0)
    
    # For each factorization method
    for(i in 1:length(factorizations)){
        
        # Extract metagenes found by factorization method
        metagenes<-factorizations[[i]][[2]][[1]]
        # Number of factors
        num.factors <- ncol(metagenes)
        # Rename columns
        colnames(metagenes)<-1:num.factors
        # Rename rows to remove "|" characters and keep only the gene name before
        rownames(metagenes)<-gsub("\\|",".",rownames(metagenes))
        rownames(metagenes)<-gsub("\\..*","",rownames(metagenes))
        # Remove duplicated gene names that could confuse fgsea
        duplicated_names <- unique(rownames(metagenes)[duplicated(rownames(metagenes))])
        metagenes <- metagenes[!(rownames(metagenes) %in% duplicated_names), ]

        # Variables
        min_pval <- numeric(0)
        path <- numeric(0)
        n <- 0
        
        # Calculate biological annotation enrichment.
        # For each factor,
        for(j in 1:num.factors){
            # Assign gene names
            rnk <- setNames(as.matrix(metagenes[,j]), rownames(metagenes))
            # Compute fgsea
            fgseaRes <- fgsea(pathways, rnk, minSize=15, maxSize=500, nperm=1000)
            # If at least one pathway is significant
            if(sum(fgseaRes$padj < pval.thr)!=0){
                # Count this factor
                n <- n+1
                # Keep min adjusted p-value
                min_pval <- rbind(min_pval, min(fgseaRes$padj))
                # Keep names of significant pathways
                path <- c(path, fgseaRes[fgseaRes$padj<pval.thr, "pathway"])
            } else {
                min_pval <- rbind(min_pval, NA)
            }
        }

        # Report number of unique significant pathways  
        if(length(path)==0){
            report_number <- rbind(report_number, 0)
        }else{
            report_number <- rbind(report_number, length(unique(path)))
        }
        # Report selectivity 
        if(length(unique(path))==0){
            report_select <- rbind(report_select, NA)
        }else{
            al<-length(unique(path))/length(path)
            fl<-length(which(!is.na(min_pval)))/length(path)
            report_select <- rbind(report_select, (al+fl)/2)
        }
        # Report number of factors associated with at least one significant pathway
        report_nnzero<-rbind(report_nnzero, n)    
    
    }
    
    out <- data.frame(selectivity=report_select, nonZeroFacs=report_nnzero, total_pathways=report_number)
#    print(out)
    return(out)
}

## Running the comparisons in cancer 对癌症进行比较

We here run the three functions defined above to perform the comparison on the cancer data. The cancer data should be organized into the data folder, each of them having the name of a different cancer type and containin the various omics data (3 omics for our test cases). Using the script povided in our github this step is automatically performed.
我们在这里运行上面定义的三个函数来对癌症数据执行比较。癌症数据应该被组织到数据文件夹中，每个数据文件夹都有不同癌症类型的名称，并且包含在不同的组学数据中（我们的测试案例有3个组学）。使用github中提供的脚本，这一步将自动执行。

In [4]:
# Load the function running the factorization, plus a support function
source("runfactorization.R")
source("log2matrix.R")

# List downloaded cancer data.
# Folder structure should be organized as discussed above.
# Exclude first result as it's the parent folder
cancers <- list.dirs(path = "../data/cancer", full.names = TRUE, recursive = TRUE)[-1]
cancer_names <- list.dirs(path = "../data/cancer", full.names = FALSE, recursive = TRUE)[-1]

# Annotation databases used for biological enrichment
path.database <- "../data/bio_annotations/c2.cp.reactome.v6.2.symbols.gmt" #REACTOME
#path.database <- "../data/bio_annotations/h.all.v6.2.symbols.gmt" #Hallmarks
#path.database <- "../data/bio_annotations/c5.all.v6.2.symbols.gmt" #GO

# Label to identify current run
tag <- format(Sys.time(), "%Y%m%d%H%M%S")
# Folder for comparison results
results_folder <- paste0("../results", tag, "/")
# Create output folder
dir.create(results_folder, showWarnings = FALSE)

# Number of factors used in the paper
num.factors <- 10

# Initialize result containers
clinical_analysis <- data.frame(
    matrix(data = NA, ncol=5, nrow=0, 
           dimnames = list(c(), c("methods", "cancer", "selectivity", "nonZeroFacs", "total_pathways"))
          ),
    stringsAsFactors = FALSE)

biological_analysis <- data.frame(
    matrix(data = NA, ncol=5, nrow=0, 
           dimnames = list(c(), c("methods", "cancer", "selectivity", "nonZeroFacs", "total_pathways"))
          ),
    stringsAsFactors = FALSE)
cancer.list <- list()

# Clinical categories to be used for clinical tests
col <- c("age_at_initial_pathologic_diagnosis",
         "gender",
         "days_to_new_tumor_event_after_initial_treatment",
         "history_of_neoadjuvant_treatment")

Loading required package: MASS

Loading required package: NMF

Loading required package: pkgmaker

Loading required package: registry

Loading required package: rngtools

Loading required package: cluster

NMF - BioConductor layer [OK] | Shared memory capabilities [NO: bigmemory] | Cores 27/28

  To enable shared memory capabilities, try: install.extras('
NMF
')

Loading required package: mclust

Package 'mclust' version 5.4.6
Type 'citation("mclust")' for citing this R package in publications.

Loading required package: InterSIM

Loading required package: tools

Loading required package: ade4


Attaching package: ‘ade4’


The following object is masked from ‘package:BiocGenerics’:

    score



Attaching package: ‘GPArotation’


The following object is masked from ‘package:NMF’:

    entropy



Attaching package: ‘MOFAtools’


The following objects are masked from ‘package:NMF’:

    featureNames, featureNames<-, predict, sampleNames, sampleNames<-


The following objects are masked f

In [5]:
print(cancers)

[1] "../data/cancer/aml"      "../data/cancer/breast"  
[3] "../data/cancer/colon"    "../data/cancer/kidney"  
[5] "../data/cancer/liver"    "../data/cancer/lung"    
[7] "../data/cancer/melanoma" "../data/cancer/ovarian" 
[9] "../data/cancer/sarcoma" 


In [6]:
# For each cancer dataset
for(i in cancers){

    print(paste0("Now analysing ", i))
    
    # Name of current cancer
    current_cancer <- basename(i)

    # If the expression and miRNA data are not log2-transformed as for those provided by XX et al.
    log2matrix(i,"exp")
    log2matrix(i,"mirna")

    # Perform factorisation
    print("Running factorisation...")
    out <- runfactorization(i, c("log_exp","methy","log_mirna"), num.factors, sep=" ", filtering="sd")
    
    # Survival analysis
    print("Running survival analysis...")
    survival <- read.table(paste0(i, "/survival"), sep="\t", header=TRUE, stringsAsFactors=FALSE)
    out_survival <- survival_comparison(out$factorizations, out$method, survival, 
                                        results_folder, current_cancer)
    
    # Clinical analysis
    print("Running clinical analysis...")
    clinical <- read.table(paste0("../data/clinical/", current_cancer), sep="\t", header=TRUE)
    out_clinical <- clinical_comparison(out$factorizations, clinical, col)   
    clinical_analysis <- rbind(clinical_analysis,
                                data.frame(methods=out$method, cancer=current_cancer, out_clinical))    

    # Biological analysis
    print("Running biological analysis...")
    out_bio <- biological_comparison(out$factorizations, path.database, pval.thr=0.05)
    biological_analysis <- rbind(biological_analysis,
                                data.frame(methods=out$method, cancer=current_cancer, out_bio))    
}
rownames(clinical_analysis) <- c()
rownames(biological_analysis) <- c()

[1] "Now analysing ../data/cancer/aml"
[1] "Running factorisation..."


Warning message in createMOFAobject(omics):
“View names are not specified in the data, renaming them to: view_1, view_2, view_3
”
Creating MOFA object from list of matrices,
 please make sure that samples are columns and features are rows...


Checking data options...

Checking training options...

Checking model options...



[1] "No output file provided, using a temporary file..."
Generating warm start... 
K=11:1234
[1] "Running survival analysis..."
[1] "Running clinical analysis..."
[1] "Running biological analysis..."
[1] "Now analysing ../data/cancer/breast"
[1] "Running factorisation..."


Warning message in createMOFAobject(omics):
“View names are not specified in the data, renaming them to: view_1, view_2, view_3
”
Creating MOFA object from list of matrices,
 please make sure that samples are columns and features are rows...


Checking data options...

Checking training options...

Checking model options...



[1] "No output file provided, using a temporary file..."
Generating warm start... 
K=11:123456
[1] "Running survival analysis..."
[1] "Running clinical analysis..."
[1] "Running biological analysis..."
[1] "Now analysing ../data/cancer/colon"
[1] "Running factorisation..."


...

456
[1] "Running survival analysis..."
[1] "Running clinical analysis..."
[1] "Running biological analysis..."
[1] "Now analysing ../data/cancer/lung"
[1] "Running factorisation..."


Warning message in createMOFAobject(omics):
“View names are not specified in the data, renaming them to: view_1, view_2, view_3
”
Creating MOFA object from list of matrices,
 please make sure that samples are columns and features are rows...


Checking data options...

Checking training options...

Checking model options...



[1] "No output file provided, using a temporary file..."
Generating warm start... 
K=11:12345
[1] "Running survival analysis..."


Warning message in max(event[who2]):
“max里所有的参数都不存在；回覆-Inf”


ERROR: Error in coxph(Surv(ordered.survival.data$Survival, ordered.survival.data$Death) ~ : No (non-missing) observations


## Saving results to file
Here the obtained results are saved to file in the Results folder.

In [ ]:
# Export results into separated tables
write.table(biological_analysis, paste0(results_folder, "results_biological_analysis.txt"), 
            sep="\t", row.names=FALSE)

write.table(clinical_analysis, paste0(results_folder, "results_clinical_analysis.txt"), 
            sep="\t", row.names=FALSE)

## Plots of the obtained results with respect to biological annotations
Plots are then created accordingly to Figure 4 and Supp Figure 2 of the paper.

In [ ]:
library(ggplot2)

In [ ]:
#tiff("BioEnr_go_fig1.tiff", units="in", width=8.95, height=6.05, res=300)
min_nonZero = min(biological_analysis[, "nonZeroFacs"]) 
max_nonZero = max(biological_analysis[, "nonZeroFacs"]) 
g <- ggplot(biological_analysis, 
            aes(x=nonZeroFacs,y=selectivity)) + 
    geom_point(aes(colour = methods, shape = cancer), size=5, alpha=.6, position=position_jitter(h=0, w=0.15))+ 
    theme_bw() + 
    scale_shape_manual(values=c(15,17,16)) + 
    scale_color_manual(values=c('#FF00FF', '#FF6E28', '#C8961E', '#FF0000', '#0000FF', '#A0A0A0', '#48D1CC', '#00FF00')) +
    ylim(floor(min((biological_analysis[,"selectivity"]*10)-.4)) / 10,
         ceiling(max((biological_analysis[,"selectivity"]*10)+.2)) / 10) +
    labs(title="Biological annotations", 
         x="# metagenes (factors) enriched in at least one annotation") +
    theme(plot.title = element_text(size=14,face="bold"),
          axis.text = element_text(size=11),
          axis.title = element_text(size=13),
          legend.text=element_text(size=10)) +
    ylab("Selectivity") + 
    labs(colour = "Methods", shape = "Cancer") +
    guides(color = guide_legend(order = 1),shape = guide_legend(order = 2),size = guide_legend(order = 3)) + 
    #scale_x_discrete() + scale_x_discrete(limits=min_nonZero:max_nonZero) + 
    scale_x_discrete(limits=min_nonZero:max_nonZero, labels = c(min_nonZero:max_nonZero));
g
#dev.off()

ggsave(paste0(results_folder, "biological_annotations.pdf"),dpi=300)
ggsave(paste0(results_folder, "biological_annotations.png"),dpi=300) 

## Plots of the obtained results with respect to clinical annotations
Plots are then created accordingly to Figure 4 and Supp Figure 2 of the paper.

In [ ]:
library(ggplot2)

In [ ]:
#tiff("BioEnr_go_fig1.tiff", units="in", width=8.95, height=6.05, res=300)
min_nonZero = min(clinical_analysis[, "nonZeroFacs"]) 
max_nonZero = max(clinical_analysis[, "nonZeroFacs"]) 
g <- ggplot(clinical_analysis, 
            aes(x=nonZeroFacs, y=selectivity)) + 
    geom_point(aes(colour = methods, shape = cancer), size=5, alpha=.6, position=position_jitter(h=0, w=0.15))+ 
    theme_bw() + 
    scale_shape_manual(values=c(15,17,16)) + 
    scale_color_manual(values=c('#FF00FF', '#FF6E28', '#C8961E', '#FF0000', '#0000FF', '#A0A0A0', '#48D1CC', '#00FF00')) +
    ylim(floor(min((clinical_analysis[,"selectivity"]*10)-.4)) / 10,
         ceiling(max((clinical_analysis[,"selectivity"]*10)+.2)) / 10) +
    labs(title="Clinical annotations", 
         x="# metagenes (factors) enriched in at least one annotation") +
    theme(plot.title = element_text(size=14,face="bold"),
          axis.text = element_text(size=11),
          axis.title = element_text(size=13),
          legend.text=element_text(size=10)) +
    ylab("Selectivity") + 
    labs(colour = "Methods",shape="Cancer") +
    guides(color = guide_legend(order = 1),shape = guide_legend(order = 2),size = guide_legend(order = 3)) + 
    #scale_x_discrete() + scale_x_discrete(limits=min_nonZero:max_nonZero) + 
    scale_x_discrete(limits=min_nonZero:max_nonZero, labels = c(min_nonZero:max_nonZero));
g
#dev.off()

ggsave(paste0(results_folder, "clinical_annotations.pdf"),dpi=300)
ggsave(paste0(results_folder, "clinical_annotations.png"),dpi=300) 